# Using the Qwen language model with AWQ quantization

Originally, this notebook contained code to run a AWQ-quantized
language model. In earlier versions, this was included in the 
`transformers` library, but it has become obsolete.

As of now (2025-11-17), it is not working anymore. AWQ is a powerful
quantization technique and has been integrated into `vllm`, which 
you will see later.